In [1]:
import numpy as np
import scipy.io as sio
import glob
from sklearn import linear_model
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
from itertools import cycle
from scipy.optimize import curve_fit
from sklearn.decomposition import PCA
from scipy.stats import sem
from scipy import stats
from scipy import ndimage
import pystan
from matplotlib import cm

%matplotlib inline

color_cycle = cycle(['g', 'b', 'c', 'm', 'y', 'k'])


import warnings
warnings.simplefilter("ignore")


import matplotlib as mpl
mpl.rcParams['font.size'] = 16
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'large'

%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

color_cycle = cycle(['g', 'b', 'c', 'm', 'y', 'k'])


import sys
sys.path.append('C:\\Users\\adamimos\\Documents\\GitHub\\ASAC\\python')
from import_rat_data import *
from behavior_analysis import *
#from decoder_functions import *

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import cv2

In [2]:
file = open("C:\\Users\\adamimos\\Documents\\DATA\\Complete_data_files\\MotionPulses\\adam0_20190729.pickle",'rb')
#file = open("C:\\Users\\adamimos\\Documents\\DATA\\Complete_data_files\\Dots8dir\\Adam0_20190301.pickle",'rb')
data = pickle.load(file)
file.close()   

In [3]:
data['completed_trials_data'].keys()

dict_keys(['was_completed', 'was_correct', 'correct_side', 'prior', 'noise', 'response_side', 'traces_stim_aligned', 'traces_resp_aligned', 'completed_inds', 'frame_info'])

In [5]:
traces = data['completed_trials_data']['traces_stim_aligned']
directions = data['completed_trials_data']['correct_side']


In [6]:
independent_variable= directions
data_to_avg = traces
av_traces = []
for x in np.unique(independent_variable):
    av_traces.append(np.mean(data_to_avg[np.where(independent_variable==x)[0],:,:],axis=0))
av_traces = np.array(av_traces)


In [7]:
import matplotlib.pylab as pl

def plot_av_dir_traces(n):
    #fig = plt.figure(figsize=(20,7))
    colors = pl.cm.RdBu(np.linspace(0,1,av_traces.shape[0]))
    #plt.subplot(1,2,1)
    for x in range(av_traces.shape[0]):
        plt.plot(np.squeeze(av_traces[x,n,:]),color = colors[x])  
    plt.plot([35,35],[0,np.max(np.ravel(av_traces[:,n,:]))],'--k')
    plt.plot([40,40],[0,np.max(np.ravel(av_traces[:,n,:]))],'--k')
    plt.plot([47,47],[0,np.max(np.ravel(av_traces[:,n,:]))],'--k')
    plt.xticks([35,43], ['poke','stim'])
    plt.xlim([20,60])

In [8]:
interact(plot_av_dir_traces,n=widgets.IntSlider(min=0,max=data_to_avg.shape[1]-1,step=1,value=0))

interactive(children=(IntSlider(value=0, description='n', max=252), Output()), _dom_classes=('widget-interact'…

<function __main__.plot_av_dir_traces(n)>

In [11]:
def plot_rasters(n):
    fig = plt.figure(figsize=(20,20))
    plt.subplot(2,2,1)
    plt.imshow(data_to_avg[np.argsort(independent_variable),n,:],aspect='auto')
    #deltas = np.where(np.diff(independent_variable[np.argsort(independent_variable)]))[0]
    #for i in range(len(deltas)):
        #plt.plot([20,80],[deltas[i],deltas[i]],'w--',alpha=0.4)
        #plt.arrow(30,deltas[i]-25,5*np.cos(i*np.pi/4),-5*np.sin(i*np.pi/4),color='r')
    plt.xlim([20,80])
    plt.ylim([0,len(independent_variable)])
    plt.plot([40,40],[0,len(independent_variable)],'w--',alpha=0.4)
    plt.plot([47,47],[0,len(independent_variable)],'w--',alpha=0.4)

In [12]:
interact(plot_rasters,n=widgets.IntSlider(min=0,max=av_traces.shape[1],step=1,value=0))

interactive(children=(IntSlider(value=0, description='n', max=253), Output()), _dom_classes=('widget-interact'…

<function __main__.plot_rasters(n)>

In [53]:
delta[0]

NameError: name 'delta' is not defined

In [15]:
data['completed_trials_data']['noise']

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1